<a href="https://colab.research.google.com/github/evcereti/Aprendiendo_ML_y_DS/blob/main/K_means_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#K-means clustering en el plano cartesiano

El objetivo de este primer notebook será utilizar K means para separar el plano cartesiano en K regiones.

##Primera aproximación a una firma

Empezamos por definir la firma de nuestra función kmeans que contenga la logica de nuestro algoritmo.

Una primera aproximación a la función a desarrollar sería definir una función kmeans que tome un numero entero k y regrese K posiciones en el mapa. Estas serán los centroides de cada región.

In [2]:
from typing import List

def kmeans(k: int) -> List[int]: pass

## Primera aproximación a los datos

Las regiones en el mapa no son aleatorias, mas bien, se escogeran según "la población" de puntos que recibiremos como entrada. Esto es analogo a escoger la frontera de paises en el globo basado en la cantidad de población de cada lugar.